In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import medmnist
from medmnist import INFO, Evaluator
from torchvision import transforms
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Constants
BATCH_SIZE = 64
LATENT_DIM = 64
EPOCHS = 20
LEARNING_RATE = 0.001
NUM_CLASSES = 8
MODEL_SAVE_PATH = './classifier_autoencoder_tissuemnist.pt'

# Select the dataset
DATA_FLAG = 'tissuemnist'
info = INFO[DATA_FLAG]
DataClass = getattr(medmnist, info['python_class'])

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# Load the dataset
train_dataset = DataClass(split='train', transform=transform, download=True)
val_dataset = DataClass(split='val', transform=transform, download=True)
test_dataset = DataClass(split='test', transform=transform, download=True)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 28 * 28),
            nn.Sigmoid(),
            nn.Unflatten(1, (1, 28, 28))
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

    def encode(self, x):
        return self.encoder(x)

# Classifier Model
class Classifier(nn.Module):
    def __init__(self, encoder, latent_dim, num_classes):
        super(Classifier, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        return self.classifier(encoded)

# Initialize models
autoencoder = Autoencoder(latent_dim=LATENT_DIM)
classifier = Classifier(autoencoder.encoder, LATENT_DIM, num_classes=NUM_CLASSES)

# Loss and optimizer
criterion_autoencoder = nn.MSELoss()
criterion_classifier = nn.CrossEntropyLoss()
optimizer_autoencoder = optim.Adam(autoencoder.parameters(), lr=LEARNING_RATE)
optimizer_classifier = optim.Adam(classifier.parameters(), lr=LEARNING_RATE)

# Training the Autoencoder
def train_autoencoder(model, train_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, _ in tqdm(train_loader, desc=f'Autoencoder Training Epoch {epoch + 1}/{epochs}'):
            optimizer.zero_grad()
            images = images.float()
            reconstructed = model(images)
            loss = criterion(reconstructed, images)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Training the Classifier
def train_classifier(model, train_loader, val_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f'Classifier Training Epoch {epoch + 1}/{epochs}'):
            optimizer.zero_grad()
            images = images.float()
            labels = labels.squeeze().long()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}')
        # Validation
        val_loss, val_acc = validate_classifier(model, val_loader, criterion)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')

# Validation function for the Classifier
def validate_classifier(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.float()
            labels = labels.squeeze().long()
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    val_loss /= len(val_loader)
    val_acc = accuracy_score(all_labels, all_preds)
    return val_loss, val_acc

# Testing the Classifier
def test_classifier(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.float()
            labels = labels.squeeze().long()
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    test_acc = accuracy_score(all_labels, all_preds)
    print(f'Test Accuracy: {test_acc:.4f}')

# Training and evaluation
if __name__ == '__main__':
    train_autoencoder(autoencoder, train_loader, optimizer_autoencoder, criterion_autoencoder, epochs=EPOCHS)
    train_classifier(classifier, train_loader, val_loader, optimizer_classifier, criterion_classifier, epochs=EPOCHS)
    test_classifier(classifier, test_loader)

    # Save the classifier model
    torch.save(classifier.state_dict(), MODEL_SAVE_PATH)
    print(f'Model saved to {MODEL_SAVE_PATH}')

Using downloaded and verified file: C:\Users\metho\.medmnist\tissuemnist.npz
Using downloaded and verified file: C:\Users\metho\.medmnist\tissuemnist.npz
Using downloaded and verified file: C:\Users\metho\.medmnist\tissuemnist.npz


Autoencoder Training Epoch 1/20: 100%|██████████| 2586/2586 [01:11<00:00, 36.35it/s]


Epoch [1/20], Loss: 0.6754


Autoencoder Training Epoch 2/20: 100%|██████████| 2586/2586 [01:24<00:00, 30.78it/s]


Epoch [2/20], Loss: 0.6735


...

Epoch [18/20], Loss: 1.3554
Validation Loss: 1.3687, Accuracy: 0.4956


Classifier Training Epoch 19/20: 100%|██████████| 2586/2586 [00:46<00:00, 55.92it/s]


Epoch [19/20], Loss: 1.3515
Validation Loss: 1.3629, Accuracy: 0.5066


Classifier Training Epoch 20/20: 100%|██████████| 2586/2586 [00:46<00:00, 55.19it/s]


Epoch [20/20], Loss: 1.3452
Validation Loss: 1.3622, Accuracy: 0.4917
Test Accuracy: 0.4912
Model saved to ./classifier_autoencoder_tissuemnist.pt
